In [ ]:
!pip install googletrans==3.1.0a0

In [1]:
import os
import json
import re
from tqdm import tqdm

import fitz

import googletrans
import pandas as pd

# Читаем данные Contract Understanding Atticus Dataset (CUAD)

Оригинал доступен по [ссылке](https://zenodo.org/record/4595826).

In [6]:
with open('../data/CUAD_data/CUADv1.json') as f:
    data = json.load(f)

In [15]:
texts = [example['paragraphs'][0]['context'] for example in data['data']]

Разбиваем тексты на абзацы, так как переводчик принимает только небольшие порции текста.

In [16]:
texts_chunks = []
for text in texts:
    texts_chunks.append([chunk for chunk in text.split('\n\n') if chunk])

In [9]:
translator = googletrans.Translator()

Переводим тексты по абзацам и добавляем пустые строки туда, откуда они были удалены

In [ ]:
results = []
for i, text_chunks in tqdm(enumerate(texts_chunks)):
    result = translator.translate(text_chunks, dest='ru', src='en')
    translated_chunks = [chunk_result.text for chunk_result in result]
    all_chunks = []
    j = 0
    for raw_chunk in texts[i].split('\n\n'):
        if raw_chunk:
            all_chunks.append(translated_chunks[j])
            j+=1
        else:
            all_chunks.append('')
    results.append('\n\n'.join(all_chunks))

Добавляем данные по типу договоров

In [56]:
contract_types_ru = []
contract_types_en = []
for i, example in tqdm(enumerate(data['data'])):
    contract_type = example['paragraphs'][0]['qas'][0]['answers'][0]['text']
    contract_type_ru = translator.translate(contract_type, dest='ru', src='en').text
    contract_types_ru.append(contract_type_ru)
    contract_types_en.append(contract_type)

510it [01:10,  7.27it/s]


Сохраняем данные в табличном формате

In [58]:
pd_data = pd.DataFrame({'text_en': texts,
                        'text_ru': results,
                        'contract_types_en': contract_types_en,
                        'contract_types_ru': contract_types_ru})

# Проставляем метки

In [2]:
filename2class = {}
for part_num in range(1, 4):
    for classname in os.listdir(f'../data/CUAD_data/full_contract_pdf/Part_{"I"*part_num}'):
        for filename in os.listdir(f'../data/CUAD_data/full_contract_pdf/Part_{"I"*part_num}/{classname}'):
            clear_filename = filename.lower()\
                                     .split('.pdf')[0]\
                                     .strip()\
                                     .strip('-')\
                                     .replace('agreement_option ', '')\
                                     .replace('é', 'e')
            clear_classname  = classname.split(' Agreement')[0]\
                                        .replace('Joint Venture _ Filing', 'Joint Venture')\
                                        .replace('Affiliate_Agreements', 'Affiliate')\
                                        .replace('License_Agreements', 'License')\
                                        .replace('Non_Compete_Non_Solicit', 'Non-Compete')
            filename2class[clear_filename] = clear_classname

In [7]:
target = []

for ex in data['data']:
    clear_filename = ex['title'].lower()\
                                .strip()\
                                .strip('-')\
                                .replace('é', 'e')
    target.append(filename2class[clear_filename])
    
pd_data['target'] = target

In [8]:
pd_data.head()

,text_en,text_ru,contract_types_en,contract_types_ru,target
0,EXHIBIT 10.6\n\n ...,ПРИЛОЖЕНИЕ 10.6\n\nДИСТРИБЬЮТОРСКОЕ СОГЛАШЕНИЕ...,DISTRIBUTOR AGREEMENT,ДИСТРИБЬЮТОРСКОЕ СОГЛАШЕНИЕ,Distributor
1,Exhibit 10.26 CONFIDENTIAL TREATMENT HAS BE...,Приложение 10.26 БЫЛО ЗАПРОШЕНО СООТВЕТСТВИЕ К...,Promotion and Distribution Agreement,Соглашение о продвижении и распространении,Promotion
2,Exhibit 10.16 SUPPLY CONTRACT Contract No: Dat...,Приложение 10.16 КОНТРАКТ ПОСТАВКИ Контракт №:...,SUPPLY CONTRACT,ДОГОВОР ПОСТАВКИ,Supply
3,1 ...,1 Приложение 10.3\n\nя-на. (LOGO) www.i-on.com...,WEB SITE HOSTING AGREEMENT,СОГЛАШЕНИЕ О РАЗМЕЩЕНИИ ВЕБ-САЙТА,Hosting
4,Exhibit 1\n\nJOINT FILING AGREEMENT\n\nThe und...,Экспонат 1\n\nСОГЛАШЕНИЕ О СОВМЕСТНОЙ ПОДАЧЕ\n...,JOINT FILING AGREEMENT,СОГЛАШЕНИЕ О СОВМЕСТНОЙ ПОДАЧЕ,Joint Venture


In [39]:
pd_data.to_csv('../data/CUAD_data/translated_data.csv')